# Pregunta 16

10 categorías que tienen la menor cantidad de contenido anónimo publicado. 

In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [2]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

sqlContext = SQLContext(sc)

21/10/29 19:19:36 WARN Utils: Your hostname, Ideapad3 resolves to a loopback address: 127.0.1.1; using 192.168.0.237 instead (on interface wlp0s20f3)
21/10/29 19:19:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/10/29 19:19:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/home/luis/anaconda3/envs/organizacionDatos/lib/python3.9/site-packages/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [3]:
wiki_contents = sqlContext.read.csv('./Wikipedia_dataset_2c2021/contents.csv', 
                                    header=True, inferSchema=True, multiLine=True)
rdd_wiki_contents = wiki_contents.rdd

categorylinks = sqlContext.read.csv('./Wikipedia_dataset_2c2021/categorylinks.csv', 
                                    header=True, inferSchema=True, multiLine=True)
rdd_categorylinks = categorylinks.rdd

## Filtro los contenidos anonimo publicado

In [4]:
rdd_anonimo = rdd_wiki_contents.filter(lambda x: x.revisor_id == None and x.revisor_ip != None)

## Creo RDD del tipo (K,V) 

Creo RDD del tipo (clave,valor) con los ID de las publicaciones anonimas para intersectarlos con el RDD de categorias.

Nota:

`cl_from`: ID del contenido

`cl_to`: Categoría a la que pertenece el contenido

In [5]:
contenido_anonimo = rdd_anonimo.map(lambda x: (x.id, 1))

In [6]:
categorylinks_contenido = rdd_categorylinks.map(lambda x: (x.cl_from,x.cl_to))

## Intersecto los RDD para obtener las categorias con contenido anonimo

Los articulos pueden tener distintas categorias, entonces en la interseccion habra una fila por cada categoria que tenga el articulo

In [7]:
categorias_anonimas = contenido_anonimo.join(categorylinks_contenido).cache()

### Vericacion de datos

Verifico que los articulos no tengan categorias repetidas.

Por ejemplo, para la publicacion con ID 1565, no debe tener 2 veces repetida la categoria 'Normas_IEC'.

Para esto se debe cumplir que el numero total de categorias para contenido anonimo debe ser igual al numero total de categorias unicas para el contenido anonimo.

In [8]:
categorias_anonimas.count()

1038768

In [9]:
categorias_anonimas.distinct().count()

1038768

## Busco 10 categorías que tienen la menor cantidad de contenido anónimo publicado. 

Cuento la cantidad de veces que se repite la categoria (que representa la cantidad de articulos anonimos con esta categoria) y las ordeno de menor a mayor.

In [10]:
categorias_anonimas.map(lambda x: (x[1][1],1)).reduceByKey(lambda x,y: x+y).takeOrdered(10, lambda x: x[1])

[('Historiadores_de_la_provincia_de_Valladolid', 1),
 ('Atletas_de_Cuba_en_los_Juegos_OlÃ\xadmpicos_de_Barcelona_1992', 1),
 ('Nacidos_en_el_cantÃ³n_de_Berna', 1),
 ('Presidentes_de_Quito', 1),
 ('Las_aventuras_del_capitÃ¡n_Alatriste', 1),
 ('Cliff_Richard', 1),
 ('Equipos_de_FÃ³rmula_3', 1),
 ('Vuelta_a_Suiza', 1),
 ('Equipos_de_voleibol_de_AndalucÃ\xada', 1),
 ('Metro_de_Brasilia', 1)]